In [23]:
import requests
from bs4 import BeautifulSoup

url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements with data-stat="score"
score_cells = soup.find_all('td', {'data-stat': 'score'})

# Extract game IDs from the <a> tags within the score_cells
game_ids = [cell.find('a')['href'].split('/')[-1] for cell in score_cells if cell.find('a')]

# Print the list of game IDs
print("List of Game IDs:")
print(game_ids)


List of Game IDs:
['Huddersfield-Town-Burnley-July-29-2022-Championship', 'Hull-City-Bristol-City-July-30-2022-Championship', 'Rotherham-United-Swansea-City-July-30-2022-Championship', 'Millwall-Stoke-City-July-30-2022-Championship', 'Luton-Town-Birmingham-City-July-30-2022-Championship', 'Blackburn-Rovers-Queens-Park-Rangers-July-30-2022-Championship', 'Wigan-Athletic-Preston-North-End-July-30-2022-Championship', 'Blackpool-Reading-July-30-2022-Championship', 'Cardiff-City-Norwich-City-July-30-2022-Championship', 'Middlesbrough-West-Bromwich-Albion-July-30-2022-Championship', 'Sunderland-Coventry-City-July-31-2022-Championship', 'Watford-Sheffield-United-August-1-2022-Championship', 'Birmingham-City-Huddersfield-Town-August-5-2022-Championship', 'Norwich-City-Wigan-Athletic-August-6-2022-Championship', 'Queens-Park-Rangers-Middlesbrough-August-6-2022-Championship', 'Preston-North-End-Hull-City-August-6-2022-Championship', 'Stoke-City-Blackpool-August-6-2022-Championship', 'Sheffield-U

In [ ]:
[cell.find('a')['href'].split('/') for cell in score_cells if cell.find('a')]

In [3]:
len(game_IDs)


NameError: name 'game_IDs' is not defined